In [1]:
import requests

In [2]:
import random

In [17]:
import time

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By

In [2]:
from bs4 import BeautifulSoup

# admin site project -----

In [34]:
url = "https://qa1-aws.oceaniacruises.com/admin/"

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

# admin site project: identify if admin contains redirect (complete) - the redirects are as follow: /admin - /admin/ - ReturnUrl=

In [35]:
with requests.Session() as session:
    response = session.get(url, headers=headers, allow_redirects=False)
    print(response.status_code, response.headers['Location'])
    # redirect_url = response.headers["location"]

302 /loginredirector.aspx?ReturnUrl=%2fadmin%2f


# admin site project: handle the redirect by comparing my url with the current one (this can later be used to switch the POST to GET and back to POST)

In [ ]:
with requests.Session() as session:
    response = session.get(url, headers=headers, allow_redirects=True)
    if response.url =! url:
        print('not the same url')

In [15]:
response = requests.get(url, headers=headers)

In [16]:
soup = BeautifulSoup(response.text, "html.parser")

In [17]:
find_form = soup.find_all('form')

In [18]:
form_list = list(find_form)

In [19]:
for i, form in enumerate(form_list):
    #if 'country' not in form.prettify().lower():
    if 'input-575-218' in form.prettify().lower():
        print(f"Form {i}:\n{form.prettify()}\n\n")

Form 2:
<form autocomplete="off" class="c2_form form-material" data-js="signin-form">
 <div class="form-group">
  <!-- Error Message -->
  <div class="errorMessage hide" data-js="data-msg-error">
   <span>
    Sorry, we donât recognize that email and password combination. Please try again or recover your password.
   </span>
  </div>
 </div>
 <div class="form-group -icon">
  <label class="label-control" for="input-575-218">
   Email Address
  </label>
  <div class="input-group">
   <div class="input-group-prepend">
    <span class="input-group-text ogi-icon-envelope">
    </span>
   </div>
   <input class="form-control" data-msg="Please enter a valid email address." data-msg-customemail="Please enter a valid email address." data-msg-email="Please enter a valid email address." id="input-575-218" name="username" placeholder="Email Address" type="text"/>
   <!-- Error Message -->
   <div class="form-validation" data-container="#tooltip-f-745-803" data-js="form-validation" data-original-

-- cruise finder page, expand the drop down list check

In [23]:
chrome_options = Options()
chrome_options.add_argument("User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
chrome_options.add_argument("--headless=new")  # New headless implementation
chrome_options.add_argument("--disable-gpu")  # Disable GPU (useful for older environments)
chrome_options.add_argument("--window-size=1920,1080")  # Set window size for headless
chrome_options.add_argument("--no-sandbox")  # Bypass sandbox (useful in CI)
chrome_options.add_argument("--disable-dev-shm-usage")  # Prevent shared memory issues

In [30]:
driver = webdriver.Chrome(options=chrome_options)

In [31]:
try:
    url = "https://preprod-aws.oceaniacruises.com/cruise-finder#sort=featured:desc&page=1&pageSize=10"
    driver.get(url)
    time.sleep(10)
    # ensure the results loaded by checking the value of view button
    element = driver.find_element(By.CLASS_NAME, 'c265_aside_item')
    is_page_loaded = "0" not in element.get_attribute("outerText")
    drop_down_class = ".btn.dropdown-toggle.btn-outline-black.btn-md" # Class of the dropdown - using CSS selector to acount for the spaces
    if is_page_loaded:
        print('Page loaded successfully')
        print(element.get_attribute("outerText")) # Print the outerText of the element
        drop_down_element = driver.find_element(By.CSS_SELECTOR, drop_down_class) # Find the dropdown - changed to CSS selector
        '''
        if drop_down_element:
            print('Dropdown found successfully')
            driver.quit()
        else:
            print('Dropdown not found')
            driver.quit()
        '''
        drop_down_element.click() # Click the dropdown
        drop_down_expanded = drop_down_element.get_attribute("aria-expanded") # get the attribute aria-expanded
        if drop_down_expanded.lower() == "true":
            print('Dropdown expanded successfully')
            print(drop_down_expanded)
            driver.quit()
        else:
            print('Dropdown did not expand successfully')
            print(drop_down_expanded)
            driver.quit()
    else:
        print('Page did not load successfully')
    print(element.get_attribute("outerText"))
    driver.quit()
except WebDriverException as e:
    print('Error loading page:', e)
    driver.quit()

Page loaded successfully
VIEW 616 CRUISES
Dropdown found successfully


MaxRetryError: HTTPConnectionPool(host='localhost', port=59250): Max retries exceeded with url: /session/8dc3ec4a1e46aeda2fd55dc50e7a5736/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109305590>: Failed to establish a new connection: [Errno 61] Connection refused'))